In [87]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [88]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [90]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model, xception.preprocess_input)

In [91]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [92]:
# bf_val_x = model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

In [104]:
# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [170]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [171]:
preds = []
nb_runs = 10
for i in range(nb_runs):
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(bf_x, y, epochs=15, batch_size=nb_batches, validation_data=(bf_val_x, y_val))
    
    pred = lm.predict(bf_x_test, batch_size=batch_size, verbose=1)
    preds.append(pred)

Found 8222 images belonging to 120 classes.
514/514 [==============================] - 126s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 1s - loss: 3.1786 - acc: 0.6123 - val_loss: 1.9997 - val_acc: 0.8470
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.4848 - acc: 0.8595 - val_loss: 1.0444 - val_acc: 0.8795
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.8484 - acc: 0.8852 - val_loss: 0.6717 - val_acc: 0.8920
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5850 - acc: 0.8955 - val_loss: 0.5127 - val_acc: 0.8980
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.4577 - acc: 0.9017 - val_loss: 0.4347 - val_acc: 0.8965
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3821 - acc: 0.9121 - val_loss: 0.3914 - val_acc: 0.8985
Epoch 7/15
8222/8222 [==============================] - 0s - loss: 0.3316 - acc: 0.9173 - val_loss: 0.3613 

10357/10357 [==============================] - 2s     
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 126s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 1s - loss: 3.1199 - acc: 0.6379 - val_loss: 1.9507 - val_acc: 0.8540
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.4485 - acc: 0.8584 - val_loss: 1.0213 - val_acc: 0.8810
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.8312 - acc: 0.8812 - val_loss: 0.6600 - val_acc: 0.8955
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5777 - acc: 0.8976 - val_loss: 0.5062 - val_acc: 0.9015
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.4520 - acc: 0.9038 - val_loss: 0.4268 - val_acc: 0.9055
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3785 - acc: 0.9121 - val_loss: 0.3833 - val_acc: 0.9080
Epoch 7/15
8222/8222 [==============================

8222/8222 [==============================] - 0s - loss: 0.1843 - acc: 0.9539 - val_loss: 0.3201 - val_acc: 0.9025
Epoch 15/15
514/514 [==============================] - 126s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 1s - loss: 3.1706 - acc: 0.6259 - val_loss: 1.9967 - val_acc: 0.8390
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 1.4849 - acc: 0.8559 - val_loss: 1.0481 - val_acc: 0.8790
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.8519 - acc: 0.8832 - val_loss: 0.6762 - val_acc: 0.8910
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.5890 - acc: 0.8958 - val_loss: 0.5159 - val_acc: 0.9000
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.4594 - acc: 0.9026 - val_loss: 0.4367 - val_acc: 0.8975
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.3849 - acc: 0.9111 - val_loss: 0.3931 - val_acc: 0.9015
Epoch 7/15
8222/8222 [===

In [172]:
pred_ensemble = np.stack(preds).mean(axis=0)

### predict

In [173]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [174]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [175]:
description = 'ensemble_10_data_aug'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [176]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [177]:
!kg submit $submission_file_name -m $description

0.31164
